In [1]:
import os
import pandas as pd
from mlwhatif.utils import get_project_root

patients = pd.read_csv(os.path.join(str(get_project_root()), "example_pipelines", "healthcare",
                                    "patients.csv"), na_values='?')
histories = pd.read_csv(os.path.join(str(get_project_root()), "example_pipelines", "healthcare",
                                     "histories.csv"), na_values='?')

data = patients.merge(histories, on=['ssn'])

In [2]:
data["full_name"] = data["first_name"] + " " + data["last_name"]
new_patient_table = data[['id', 'first_name', 'last_name', 'race', 'county', 'num_children', 'income', 'age_group', 'full_name']]
new_patient_table.to_csv('patients.csv', index=False, na_rep='?')
new_histories_table = data[['full_name', 'smoker', 'complications']]
new_histories_table.to_csv('histories.csv', index=False, na_rep='?')

In [3]:
import os
from mlwhatif.utils import get_project_root
from example_pipelines.healthcare import custom_monkeypatching
from mlwhatif import PipelineAnalyzer

HEALTHCARE_PY = os.path.join(str(get_project_root()), "experiments", "end_to_end", "datasets", "healthcare", "healthcare.py")
HEALTHCARE_PNG = os.path.join(str(get_project_root()), "experiments", "end_to_end", "datasets", "healthcare", "healthcare-dag")

analysis_result = PipelineAnalyzer \
    .on_pipeline_from_py_file(HEALTHCARE_PY) \
    .add_custom_monkey_patching_module(custom_monkeypatching) \
    .execute()

analysis_result.save_original_dag_to_path(HEALTHCARE_PNG)
# To check if fuzzy merge is as expensive as we want

INFO:Running instrumented original pipeline...
INFO:Importing libraries and monkey-patching them... (Imports are slow if not in sys.modules cache yet!)
INFO:---RUNTIME: Importing and monkey-patching took 3387.5012397766113 ms
INFO:---RUNTIME: Original pipeline execution took 1658.3688259124756 ms (excluding imports and monkey-patching)
INFO:Starting execution of 0 what-if analyses...
INFO:Estimated runtime of original DAG is 1583.6586952209473ms
INFO:Performing Multi-Query Optimization
INFO:Estimated unoptimized what-if runtime is 0ms
INFO:Estimated optimised what-if runtime is 0ms
INFO:Estimated optimisation runtime saving is 0ms
INFO:---RUNTIME: Multi-Query Optimization took 4.818916320800781 ms
INFO:Executing generated plans
INFO:---RUNTIME: Execution took 0.0209808349609375 ms
INFO:Done!
